In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import trimesh
import pyrender
import h5py
import numpy as np
import torch
import os, os.path as osp
from tqdm import tqdm

from datasets.scannet.utils_3d import ProjectionHelper, adjust_intrinsic, make_intrinsic, load_pose

In [ ]:
def get_scan_name(scene_id, scan_id):
    return f'scene{str(scene_id).zfill(4)}_{str(scan_id).zfill(2)}'

In [ ]:
subvol_size = (32, 32, 32)

In [ ]:
data_dir = Path('/mnt/data/scannet/backproj')
fname = 'train100.h5'
f = h5py.File(data_dir / fname, 'r')

In [ ]:
print(f.keys())
print(f['x'].shape)

In [ ]:
ndx = 99999
w2g, sceneid, scanid, frames = f['world_to_grid'][ndx], f['scene_id'][ndx], f['scan_id'][ndx], f['frames'][ndx]
print(w2g)
print(sceneid, scanid)
print(frames)
g2w = torch.inverse(torch.Tensor(w2g)).numpy()

## Draw one chunk and its corresponding camera pose

In [ ]:
scene = trimesh.scene.scene.Scene() 

scan_name = get_scan_name(sceneid, scanid)
print(scan_name)
root = Path('/mnt/data/scannet/scans')
scan_path = root / f'{scan_name}/{scan_name}_vh_clean_2.ply'
print(scan_path)
scan = trimesh.load(scan_path)
print(scan)
print('Add scan')
scene.add_geometry(scan)

box = trimesh.creation.box(subvol_size, g2w)
print('Add box')
scene.add_geometry(box)

intrinsic = make_intrinsic(1170.187988, 1170.187988, 647.75, 483.75)
intrinsic = adjust_intrinsic(intrinsic, [1296, 968], (40, 30))

pose_path = root / scan_name / 'pose' / f'{frames[0]}.txt'
pose = load_pose(pose_path).numpy()
focal = (intrinsic[0, 0], intrinsic[1, 1])
cam = trimesh.scene.Camera(name=f'{scan_name}_{frames[0]}', resolution=(40, 30), focal=focal, z_near=0.4, z_far=4.0) 
camball, campath = trimesh.creation.camera_marker(cam, marker_height=4, origin_size=0.05)
camball.apply_transform(pose)
campath.apply_transform(pose)
campath.colors = np.ones((5, 3)) * np.array((0, 255, 0))
print('Add cam')
scene.add_geometry(camball)
scene.add_geometry(campath)

axes = trimesh.creation.axis(axis_radius=0.1, axis_length=10)
print('Add axes')
scene.add_geometry(axes)

scene.show()

## Draw all the cameras in a scene

In [ ]:
# draw all cameras in a scene
scene = trimesh.scene.scene.Scene() 

scan_name = get_scan_name(sceneid, scanid)
print(scan_name)
root = Path('/mnt/data/scannet/scans')
scan_path = root / f'{scan_name}/{scan_name}_vh_clean_2.ply'
print(scan_path)
scan = trimesh.load(scan_path)
print(scan)
print('Add scan')
scene.add_geometry(scan)

box = trimesh.creation.box(subvol_size, g2w)
print('Add box')
scene.add_geometry(box)

intrinsic = make_intrinsic(1170.187988, 1170.187988, 647.75, 483.75)
intrinsic = adjust_intrinsic(intrinsic, [1296, 968], (40, 30))
pose_files = sorted(os.listdir(root / scan_name / 'pose'), key=lambda f: int(osp.splitext(f)[0]))
pose_indices = range(0, len(pose_files), frame_skip)
print('Add poses:', len(pose_indices))
frame_skip = 40
focal = (intrinsic[0, 0], intrinsic[1, 1]) 
for ndx, pose_ndx in enumerate(tqdm(pose_indices)):
    pose_path = root / scan_name / 'pose' / pose_files[pose_ndx]
    pose = load_pose(pose_path).numpy()
    
    cam = trimesh.scene.Camera(name=f'{scan_name}_{frames[0]}', resolution=(40, 30), focal=focal, z_near=0.4, z_far=4.0) 
    camball, campath = trimesh.creation.camera_marker(cam, marker_height=4, origin_size=0.05)
    
    camball.apply_transform(pose)
    campath.apply_transform(pose)
    campath.colors = np.zeros((5, 3))
    scene.add_geometry(camball)
    scene.add_geometry(campath)

# draw the best pose in a different color
pose_path = root / scan_name / 'pose' / f'{frames[0]}.txt'
pose = load_pose(pose_path).numpy()
focal = (intrinsic[0, 0], intrinsic[1, 1])
cam = trimesh.scene.Camera(name=f'{scan_name}_{frames[0]}', resolution=(40, 30), focal=focal, z_near=0.4, z_far=4.0) 
camball, campath = trimesh.creation.camera_marker(cam, marker_height=4, origin_size=0.05)
camball.apply_transform(pose)
campath.apply_transform(pose)
# green
campath.colors = np.ones((5, 3)) * np.array((0, 255, 0))
print('Add cam')
scene.add_geometry(camball)
scene.add_geometry(campath)

axes = trimesh.creation.axis(axis_radius=0.1, axis_length=10)
print('Add axes')
scene.add_geometry(axes)

scene.show()